> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [17]:
import pandas as pd

# Carregando o arquivo CSV
df = pd.read_csv("salaries.csv")

# Exibindo as primeiras linhas do DataFrame
print(df.head())

# Informações gerais sobre o DataFrame
print("\nInformações sobre o DataFrame:")
print(df.info())

  FIRST NAME   LAST NAME SEX         DOJ CURRENT DATE DESIGNATION   AGE  \
0     TOMASA       ARMEN   F   5-18-2014   01-07-2016     Analyst  21.0   
1      ANNIE         NaN   F         NaN   01-07-2016   Associate   NaN   
2      OLIVE        ANCY   F   7-28-2014   01-07-2016     Analyst  21.0   
3     CHERRY     AQUILAR   F  04-03-2013   01-07-2016     Analyst  22.0   
4       LEON  ABOULAHOUD   M  11-20-2014   01-07-2016     Analyst   NaN   

   SALARY        UNIT  LEAVES USED  LEAVES REMAINING  RATINGS  PAST EXP  
0   44570     Finance         24.0               6.0      2.0         0  
1   89207         Web          NaN              13.0      NaN         7  
2   40955     Finance         23.0               7.0      3.0         0  
3   45550          IT         22.0               8.0      3.0         0  
4   43161  Operations         27.0               3.0      NaN         3  

Informações sobre o DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2639 entries, 0 to 2638

#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [24]:
from sqlalchemy import (
    Column,
    Integer,
    String,
    Float,
    Date,
    Enum
)
from sqlalchemy.ext.declarative import declarative_base
import enum

# Base declarativa para o ORM
Base = declarative_base()

# Definição das classes Enum para SEX, DESIGNATION e UNIT
class SexEnum(enum.Enum):
    F = 'Feminino'
    M = 'Masculino'

class DesignationEnum(enum.Enum):
    ANALYST = 'Analista'
    SENIOR_ANALYST = 'Analista Sênior'
    MANAGER = 'Gerente'

class UnitEnum(enum.Enum):
    IT = 'TI'
    FINANCE = 'Finanças'
    MARKETING = 'Marketing'

# Definição do modelo ORM
class Salary(Base):
    __tablename__ = 'salaries'

    id = Column(Integer, primary_key=True, autoincrement=True)  # Chave primária
    first_name = Column(String, nullable=True)                # Primeiro nome
    last_name = Column(String, nullable=True)                 # Sobrenome
    sex = Column(Enum(SexEnum), nullable=True)                # Sexo
    doj = Column(Date, nullable=True)                         # Data de ingresso
    current_date = Column(Date, nullable=True)                # Data atual
    designation = Column(Enum(DesignationEnum), nullable=True) # Designação
    age = Column(Integer, nullable=True)                      # Idade
    salary = Column(Float, nullable=True)                     # Salário
    unit = Column(Enum(UnitEnum), nullable=True)              # Unidade
    leaves_used = Column(Integer, nullable=True)              # Licenças usadas
    leaves_remaining = Column(Integer, nullable=True)         # Licenças restantes
    ratings = Column(Float, nullable=True)                    # Avaliações
    past_exp = Column(Float, nullable=True)                   # Experiência anterior

/tmp/ipykernel_7291/3015629240.py:13: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [25]:
from sqlalchemy import create_engine

# Estabelecendo a conexão com o banco de dados SQLite chamado "salarios"
engine = create_engine("sqlite:///salarios.db")

print("Conexão com o banco de dados 'salarios' estabelecida com sucesso.")

Conexão com o banco de dados 'salarios' estabelecida com sucesso.


#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [26]:
# Criando todas as tabelas definidas no modelo ORM no banco de dados 'salarios'
Base.metadata.create_all(engine)

print("Tabelas criadas no banco de dados 'salarios' com sucesso.")

Tabelas criadas no banco de dados 'salarios' com sucesso.


#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [27]:
# Renomeando as colunas do DataFrame para corresponder aos nomes do modelo ORM
df.rename(columns={
    "FIRST NAME": "first_name",
    "LAST NAME": "last_name",
    "SEX": "sex",
    "DOJ": "doj",
    "CURRENT DATE": "current_date",
    "DESIGNATION": "designation",
    "AGE": "age",
    "SALARY": "salary",
    "UNIT": "unit",
    "LEAVES USED": "leaves_used",
    "LEAVES REMAINING": "leaves_remaining",
    "RATINGS": "ratings",
    "PAST EXP": "past_exp"
}, inplace=True)

# Inserindo os dados no banco de dados
df.to_sql('salaries', con=engine, if_exists='append', index=False)

print("Dados inseridos na tabela 'salaries' com sucesso.")

Dados inseridos na tabela 'salaries' com sucesso.


#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [29]:
# Forma 1: Executando a query SQL diretamente com engine.connect()
with engine.connect() as connection:
    result = connection.exec_driver_sql(
        """
        SELECT DESIGNATION, 
               MIN(SALARY / 12) AS min_monthly_salary,
               MAX(SALARY / 12) AS max_monthly_salary,
               AVG(SALARY / 12) AS avg_monthly_salary
        FROM salaries
        GROUP BY DESIGNATION;
        """
    )
    print("Forma 1 - Resultado da Query via engine.connect():")
    for row in result:
        print(row)

Forma 1 - Resultado da Query via engine.connect():
('Analyst', 3333.4166666666665, 4165.0, 3751.6759876859915)
('Associate', 5846.166666666667, 8300.25, 7266.915094339625)
('Director', 17832.25, 32342.666666666668, 23914.265625000004)
('Manager', 8343.666666666666, 12407.5, 10522.716049382716)
('Senior Analyst', 4170.333333333333, 5830.5, 4991.778792134831)
('Senior Manager', 12614.416666666666, 16631.416666666668, 14888.689516129034)


In [30]:
# Forma 2: Executando a query SQL com Pandas (read_sql_query)
query = """
    SELECT DESIGNATION, 
           MIN(SALARY / 12) AS min_monthly_salary,
           MAX(SALARY / 12) AS max_monthly_salary,
           AVG(SALARY / 12) AS avg_monthly_salary
    FROM salaries
    GROUP BY DESIGNATION;
"""
df_result = pd.read_sql_query(query, con=engine.connect())
print("Forma 2 - Resultado da Query via Pandas (read_sql_query):")
print(df_result)

Forma 2 - Resultado da Query via Pandas (read_sql_query):
      designation  min_monthly_salary  max_monthly_salary  avg_monthly_salary
0         Analyst         3333.416667         4165.000000         3751.675988
1       Associate         5846.166667         8300.250000         7266.915094
2        Director        17832.250000        32342.666667        23914.265625
3         Manager         8343.666667        12407.500000        10522.716049
4  Senior Analyst         4170.333333         5830.500000         4991.778792
5  Senior Manager        12614.416667        16631.416667        14888.689516


In [33]:
from sqlalchemy import func, select, literal_column
from sqlalchemy.orm import Session

# Consulta ajustada para tratar os valores de Enum como strings diretamente
with Session(engine) as session:
    stmt = (
        select(
            literal_column("designation").label("designation"),  # Tratar como string
            func.min(Salary.salary / 12).label("min_monthly_salary"),
            func.max(Salary.salary / 12).label("max_monthly_salary"),
            func.avg(Salary.salary / 12).label("avg_monthly_salary"),
        )
        .group_by(literal_column("designation"))
    )
    result = session.execute(stmt)
    print("Forma 3 - Resultado da Query via SQLAlchemy (select + Session):")
    for row in result:
        print(row)

Forma 3 - Resultado da Query via SQLAlchemy (select + Session):
('Analyst', 3333.4166666666665, 4165.0, 3751.6759876859915)
('Associate', 5846.166666666667, 8300.25, 7266.915094339625)
('Director', 17832.25, 32342.666666666668, 23914.265625000004)
('Manager', 8343.666666666666, 12407.5, 10522.716049382716)
('Senior Analyst', 4170.333333333333, 5830.5, 4991.778792134831)
('Senior Manager', 12614.416666666666, 16631.416666666668, 14888.689516129034)
